<a id='top'></a>

# Constants and calculations

This notebook shows some fundamental building blocks when creating calculatio in MASTIX Studio:
- [**Constants**](#constants). The inputs to functions in MASTIX Studio. How they are created, how they can be labelled and grouped, and how they can be formatted.
    

-  [**Calculations and derivatives**](#calculations).
Defining calculations on different parts of the computation graph (?)
-  [**Functions**](#functions).
    Using pre-built functions from MASTIX Studio and creating custom funtions directly in Python.
    
    ***

### Loading libraries

In [1]:
import sys

sys.path.append("C:\\Users\\JensSvensson\\OneDrive - Mastix\\Mastix Studio Versions\\Latest")

from pythonnet import load

load("coreclr")

import clr

clr.AddReference("MastixKernel")
clr.AddReference("MastixXml")
clr.AddReference("MastixJson")

In [2]:
import os

from System.Collections.Generic import *
from System.Diagnostics import Stopwatch
from System import AssemblyLoadEventHandler

from Quant.Differentiation import *
from Quant.Calendars import *
from Quant.Collections.Generic import *
from Quant.Currencies import *
from Quant.Formats import *
from Quant.Common import *
from Quant.Interpolations import *
from Quant.Black import *
from Quant.Distributions import *

<a id='constants'></a>

[**Go to top of notebook**](#top)

## Constants

Derivatives — or sensitivities — of a function are evaluated at a single set of values for the independent variables. For a particular evaluation of the function, the inputs are constants. Hence, the inputs to a function are referred to as constants. 

Sensitivities in MASTIX Studio are with respect to the inputs of constants. This section provides a basic overview of how constants are created, how they can be grouped, and how they can be formatted for output.

The difference between 

<div align="center">
  <img src="https://github.com/mastixstudio/mastixstudio/blob/main/assets/basic-operation-constants.png?raw=true" alt="Basic operation" style="width: 30%;">
  <br/>
  <i>Figure 2: Basic operation on two constants.</i>
  <br/>
  <br/>
</div>

 ### Labeling and formatting of constants
 
 A *simple* constant is a constant without a label or name, it can be visible or hidden when retrieving derivatives. 
 A *constant* has a label and can be equipped with formatting for output of the constant itself as well as when it is part of a derivative. 
 In the example below, the constant is output in percentage format, and hence multiplied by 100. A derivative it will be output as basis points, and the constant is hence multiplied by 10000.

In [3]:
a = SimpleConstant(2.93)
b = SimpleConstant(1.39, True)
c = Constant(1.23, "Euribor/6M.5Y")

constantNumberFormat = PlainNumberFormat(2, ".", " ")

d = Constant(0.054, "Euribor/6M.5Y", "%", 100.0, "bp", 10000, constantNumberFormat)

print(d.FormatConstant(True))

5.40 %


### Grouping of constants
When dealing with large calculations it is convenient to be able to group derivatives with respect to additional attributes. Constants can therefore be associated with *groups*, which can be defined according to (*Group name*, *Attribute value*) (?). A constants can be associated with any number of groups. Groups and their common properties are collected in *extensions* which can then be used for any number of constants. 

In [4]:
group1 = ConstantGroupDate("TradeDate", Date(2022, 7, 18))
group2 = ConstantGroupDate("TradeDate", Date(2022, 7, 19))
group3 = ConstantGroupString("ReferenceIndex", "Stibor/3M")
group4 = ConstantGroupString("ReferenceIndex", "Euribor/6M")

extension1 = ConstantExtensionUnitGroups("%", 100.0, "bp", 1.0/10000, constantNumberFormat, None, group1, group3)
extension2 = ConstantExtensionUnitGroups("%", 100.0, "bp", 1.0/10000, constantNumberFormat, None, group2, group3)
extension3 = ConstantExtensionUnitGroups("%", 100.0, "bp", 1.0/10000, constantNumberFormat, None, group1, group4)
extension4 = ConstantExtensionUnitGroups("%", 100.0, "bp", 1.0/10000, constantNumberFormat, None, group2, group4)

print(extension1) # Raw format.

Unit Groups (%, 100, Plain Number Format (1, 2, .,  , 0), bp, 0,0001, Group (TradeDate, 2022-07-18, Short Date Format), Group (ReferenceIndex, Stibor/3M, Plain String Format), Plain String Format)


<a id='calculations'></a>

[**Go to top of notebook**](#top)

## Calculations and derivatives


<img src="https://github.com/mastixstudio/mastixstudio/blob/main/assets/computation-graph-subgraph-collapsed.png?raw=true" alt="Alternative text" style="width: 40%;" />

In [5]:
# Create a couple of constants to use below.

c0 = Constant( 0.24, "Stibor/3M.1Y", extension1)
c1 = Constant( 1.24, "Stibor/3M.2Y", extension1)
c2 = Constant( 2.84, "Stibor/3M.3Y", extension2)
c3 = Constant(-4.26, "Stibor/3M.4Y", extension2)
c4 = Constant( 0.92, "Stibor/3M.5Y", extension2)
c5 = Constant( 3.12, "Euribor/6M.1Y", extension3)
c6 = Constant( 7.22, "Euribor/6M.2Y", extension3)
c7 = Constant( 9.34, "Euribor/6M.3Y", extension4)
c8 = Constant(-2.61, "Euribor/6M.4Y", extension4)
c9 = Constant( 1.23, "Euribor/6M.5Y", extension4)

print(c0) # Raw format.

Constant: 0,24 (Stibor/3M.1Y, %, 100, Plain Number Format (1, 2, .,  , 0), bp, 0,0001, Group (TradeDate, 2022-07-18, Short Date Format), Group (ReferenceIndex, Stibor/3M, Plain String Format))


# Calculate on high-performance framework.

In [6]:


tape = OuterTape()

r1 = Mathematics.Multiplication(tape, OuterReverse(c1), OuterReverse(c2))
d1 = r1.AccumulateDerivatives()

print(r1) # Raw format.
print(d1) # Raw format.

Outer: 3,5216, Height: 0
Derivatives (3.52, 2, 2)


Calculate outside high-performance framework (But with some other features instead). Note that there is no tape here.

In [7]:
r2 = Mathematics.Multiplication(c1, c2)
d2 = r2.AccumulateDerivatives()

print(r2) # Raw format.
print(d2) # Raw format.

Interior: 3,5216 (2)
Derivatives (3.52, 2, 2)


Calculate outside high-performance framework as real operators (Can for example be saved as operators and recalculated on load). Note the prefix operator

In [8]:
r3 = Mathematics.OperatorMultiplication(c1, c2)
d3 = r3.AccumulateDerivatives()

print(r3) # Raw format. Not that this is i multiplication node compared to the above which is a plain binary node.
print(d3) # Raw format.

Interior: 3,5216 (Multiplication, 2)
Derivatives (3.52, 2, 2)


Calculations can be joined (in principle) arbitrarily. The only exception is that results on the high-performance framework can not be used as inputs to calculations outside the high-performance framework.

In [9]:
r4 = Mathematics.Addition(r3, c3)
d4 = r4.AccumulateDerivatives()

print(r4) # Raw format.
print(d4) # Raw format.

Interior: -0,7384 (2)
Derivatives (-0.74, 3, 3)


A larger calculation using constants as well as previous calculated results (except r1 since it is on the high-performance framework).

In [10]:
valueNumberFormat = PlainNumberFormat(2, ".", " ")
derivativeNumberFormat = PlainNumberFormat(3, ".", " ")

r5 = Mathematics.Multiplication(c0, c1, c2, c3, r2, r3, r4, c4, c4, c4, c6)
d5 = r5.AccumulateDerivatives("SEK", 100.0, valueNumberFormat, derivativeNumberFormat, False)

print(r5) # Raw format.
print(d5) # Raw format.

Interior: 185,3673682400455 (11)
Derivatives (18 536.74 SEK, 6, 6)


However if we do the calculation on the high-performance framework we can include r1 as well. Note the tape.

In [11]:
r5t = Mathematics.Multiplication(tape, OuterReverse(c0),  OuterReverse(c7),  OuterReverse(c8),  OuterReverse(c9),  r1)
d5t = r5.AccumulateDerivatives("SEK", 100.0, valueNumberFormat, derivativeNumberFormat, False)

print(r5t) # Raw format.
print(d5t) # Raw format.

Outer: -25,342167783167998, Height: 1
Derivatives (18 536.74 SEK, 6, 6)


Show result in raw format ....

In [12]:
print(d5.Value)

for derivative in d5:
    print(derivative)

# ... or nicely formatted.

print("{0:16} : {1:>13}".format("Value", d5.FormatValue(True)))

for derivative in d5:
    print("{0:16} : {1:>13}" .format(derivative.Item1.IdNullable, d5.FormatDerivative(derivative, True)))

185.3673682400455
(Constant: 0,24 (Stibor/3M.1Y, %, 100, Plain Number Format (1, 2, .,  , 0), bp, 0,0001, Group (TradeDate, 2022-07-18, Short Date Format), Group (ReferenceIndex, Stibor/3M, Plain String Format)), 772,3640343335229)
(Constant: 1,24 (Stibor/3M.2Y, %, 100, Plain Number Format (1, 2, .,  , 0), bp, 0,0001, Group (TradeDate, 2022-07-18, Short Date Format), Group (ReferenceIndex, Stibor/3M, Plain String Format)), -264,4819770174347)
(Constant: 2,84 (Stibor/3M.3Y, %, 100, Plain Number Format (1, 2, .,  , 0), bp, 0,0001, Group (TradeDate, 2022-07-19, Short Date Format), Group (ReferenceIndex, Stibor/3M, Plain String Format)), -115,47804630338698)
(Constant: -4,26 (Stibor/3M.4Y, %, 100, Plain Number Format (1, 2, .,  , 0), bp, 0,0001, Group (TradeDate, 2022-07-19, Short Date Format), Group (ReferenceIndex, Stibor/3M, Plain String Format)), -294,5526978173349)
(Constant: 0,92 (Stibor/3M.5Y, %, 100, Plain Number Format (1, 2, .,  , 0), bp, 0,0001, Group (TradeDate, 2022-07-19, Sho

Derivatives can be accessed in numerous ways, for example

In [13]:
constants = d5.GetConstants()

for constant in constants:
    print("{0:14} : {1:>12} : {2:>16}".format(constant.IdNullable, constant.FormatConstant(True), d5.FormatDerivative(constant, d5[constant], True)))

derivatives = d5.GetDerivatives()

for derivative in derivatives:
    print("{0:14} : {1:>12} : {2:>16}".format(derivative.Item1.IdNullable, derivative.Item1.FormatConstant(True), d5.FormatDerivative(derivative, True)))


Stibor/3M.1Y   :      24.00 % :     7.724 SEK/bp
Stibor/3M.2Y   :     124.00 % :    -2.645 SEK/bp
Stibor/3M.3Y   :     284.00 % :    -1.155 SEK/bp
Stibor/3M.4Y   :    -426.00 % :    -2.946 SEK/bp
Stibor/3M.5Y   :      92.00 % :     7.142 SEK/bp
Euribor/6M.2Y  :     722.00 % :     0.257 SEK/bp
Stibor/3M.1Y   :      24.00 % :     7.724 SEK/bp
Stibor/3M.2Y   :     124.00 % :    -2.645 SEK/bp
Stibor/3M.3Y   :     284.00 % :    -1.155 SEK/bp
Stibor/3M.4Y   :    -426.00 % :    -2.946 SEK/bp
Stibor/3M.5Y   :      92.00 % :     7.142 SEK/bp
Euribor/6M.2Y  :     722.00 % :     0.257 SEK/bp


or if preferred filtered and sorted.

In [14]:
f1 = OrDerivativeFilter(DerivativeIdWildcardFilter(Wildcard("*2y*", True, True)), DerivativeIdWildcardFilter(Wildcard("*3y*", True, True)))

s1 = DerivativeGroupDateSorter("TradeDate", True)

derivatives = d5.GetDerivatives(f1, s1)

for derivative in derivatives:
    print(derivative)

(Constant: 2,84 (Stibor/3M.3Y, %, 100, Plain Number Format (1, 2, .,  , 0), bp, 0,0001, Group (TradeDate, 2022-07-19, Short Date Format), Group (ReferenceIndex, Stibor/3M, Plain String Format)), -115,47804630338698)
(Constant: 1,24 (Stibor/3M.2Y, %, 100, Plain Number Format (1, 2, .,  , 0), bp, 0,0001, Group (TradeDate, 2022-07-18, Short Date Format), Group (ReferenceIndex, Stibor/3M, Plain String Format)), -264,4819770174347)
(Constant: 7,22 (Euribor/6M.2Y, %, 100, Plain Number Format (1, 2, .,  , 0), bp, 0,0001, Group (TradeDate, 2022-07-18, Short Date Format), Group (ReferenceIndex, Euribor/6M, Plain String Format)), 25,674150725768076)


We can also do some basic arithmetics on the results ... keeping value and all derivatives up to date ....

In [15]:
d6 = Result.Addition(d4, d5)

print(d4.Value)
print(d5.Value)
print(d6.Value)

print()
print("*************************************************")
print()

# ... and look at the derivatives

derivatives = d6.GetDerivatives()

for derivative in derivatives:
    print(derivative)

-0.7384
185.3673682400455
184.62896824004548

*************************************************

(Constant: -4,26 (Stibor/3M.4Y, %, 100, Plain Number Format (1, 2, .,  , 0), bp, 0,0001, Group (TradeDate, 2022-07-19, Short Date Format), Group (ReferenceIndex, Stibor/3M, Plain String Format)), -293,5526978173349)
(Constant: 1,24 (Stibor/3M.2Y, %, 100, Plain Number Format (1, 2, .,  , 0), bp, 0,0001, Group (TradeDate, 2022-07-18, Short Date Format), Group (ReferenceIndex, Stibor/3M, Plain String Format)), -261,6419770174347)
(Constant: 2,84 (Stibor/3M.3Y, %, 100, Plain Number Format (1, 2, .,  , 0), bp, 0,0001, Group (TradeDate, 2022-07-19, Short Date Format), Group (ReferenceIndex, Stibor/3M, Plain String Format)), -114,23804630338698)
(Constant: 0,24 (Stibor/3M.1Y, %, 100, Plain Number Format (1, 2, .,  , 0), bp, 0,0001, Group (TradeDate, 2022-07-18, Short Date Format), Group (ReferenceIndex, Stibor/3M, Plain String Format)), 772,3640343335229)
(Constant: 0,92 (Stibor/3M.5Y, %, 100, Pla

We can also inspect the legs of nodes.

In [16]:
legs = r4.Trees

for leg in legs:
    print(leg)

print()
print("*************************************************")
print()

(Interior: 3,5216 (Multiplication, 2), 1)
(Constant: -4,26 (Stibor/3M.4Y, %, 100, Plain Number Format (1, 2, .,  , 0), bp, 0,0001, Group (TradeDate, 2022-07-19, Short Date Format), Group (ReferenceIndex, Stibor/3M, Plain String Format)), 1)

*************************************************



Expressions can also be evaluated on or outside the high-performance framework.

In [17]:
expression = "-cos(({0}*{1}*{2}/{3}*5+9*{4})*exp(2))"

r6 = Parser.Parse(tape, expression, OuterReverse(c1), OuterReverse(r2), OuterReverse(c3), OuterReverse(r4), OuterReverse(c5))
d6 = r6.AccumulateDerivatives()

print(r6)
print(d6)

Outer: -0,548658282371793, Height: 7
Derivatives (-0.55, 4, 4)


A comment on nodes
In order for a node to be included on the high-performance framework it needs to be an OuterReverse. However, both Interior and Constant nodes can be cast to an OuterReverse.
For the time beeing, they can not be auto-casted.
Outside the the high-performance framework, nodes should be Interior nodes (and Constants are actually Interior nodes so no cast is needed here.)

We can also the calculation tree as a DOT-file to disk ...

In [18]:
nodesSaved = CalculationTree(r6, PlainNumberFormat(2, ".", " ")).SaveToDotFile("c:\\temp\\nodes.dot", True, True, True, True)

print("{0} nodes saved.".format(nodesSaved))

24 nodes saved.


The nodes are stored as a DOT language file which can be converted
to pictures with tools understanding this DOT langauage. An example of such a
tool is graphviz (https://graphviz.org/). With graphviz, A DOT file can, for example, be translated
minto a pdf using "dot -T pdf -o dot.pdf nodes.dot"

<a id='functions'></a>

[**Go to top of notebook**](#top)

## Functions

Black Scholes as well as many, may more functions are available directly as

In [19]:
spot = Constant(123, "Spot", "SEK", 1.0, "SEK", 1.0, constantNumberFormat)
strike = Constant(119, "Strike", "SEK", 1.0, "SEK", 1.0, constantNumberFormat)
rate = Constant(0.05, "Rate", "%", 100.0, "bp", 1/100.0, constantNumberFormat)
volatility = Constant(0.4, "Volatility", "%", 100.0, "bp", 1/100.0, constantNumberFormat)
expiry = Constant(0.6, "Expiry", "Years", 1.0, "Day", 1/365.0, constantNumberFormat)

call = BlackScholes.Call(spot, strike, rate, volatility, expiry).AccumulateDerivatives()

print(call.Value)

for derivative in call:
    print(derivative)

18.739392302582672
(Constant: 123 (Spot, SEK, 1, Plain Number Format (1, 2, .,  , 0), SEK, 1), 0,6399957344419552)
(Constant: 119 (Strike, SEK, 1, Plain Number Format (1, 2, .,  , 0), SEK, 1), -0,504034311208217)
(Constant: 0,05 (Rate, %, 100, Plain Number Format (1, 2, .,  , 0), bp, 0,01), 35,98804982026669)
(Constant: 0,4 (Volatility, %, 100, Plain Number Format (1, 2, .,  , 0), bp, 0,01), 35,64435338536214)
(Constant: 0,6 (Expiry, Years, 1, Plain Number Format (1, 2, .,  , 0), Day, 0,0027397260273972603), 14,880455280142936)


...but can also be built using other functions or parsed from expressions as this somewhat example shows:

In [20]:
def SemiParsedCall(spot, strike, rate, vol, expiry):
    
    num = Parser.Parse("ln({0}/{1})+({2}+({3}*{3}/2))*{4}", spot, strike, rate, vol, expiry)

    den = Parser.Parse("{0}*sqrt({1})", vol, expiry)

    d1 = Mathematics.Division(num, den)

    d2 = Mathematics.Subtraction(d1, den)

    p1 = StandardNormal.CumulativeDistribution(d1)
    p2 = StandardNormal.CumulativeDistribution(d2)

    return Parser.Parse("{0}*{1}-{2}*{3}*exp(-{4}*{5})", p1, spot, p2, strike, rate, expiry)


semiParsedCall = SemiParsedCall(spot, strike, rate, volatility, expiry).AccumulateDerivatives()

print(semiParsedCall.Value)

for derivative in semiParsedCall:
    print(derivative)

18.73939230258268
(Constant: 119 (Strike, SEK, 1, Plain Number Format (1, 2, .,  , 0), SEK, 1), -0,5040343112082167)
(Constant: 123 (Spot, SEK, 1, Plain Number Format (1, 2, .,  , 0), SEK, 1), 0,639995734441955)
(Constant: 0,6 (Expiry, Years, 1, Plain Number Format (1, 2, .,  , 0), Day, 0,0027397260273972603), 14,880455280142936)
(Constant: 0,05 (Rate, %, 100, Plain Number Format (1, 2, .,  , 0), bp, 0,01), 35,98804982026667)
(Constant: 0,4 (Volatility, %, 100, Plain Number Format (1, 2, .,  , 0), bp, 0,01), 35,644353385362145)


Finally - solve for strike giving a specified call value.

In [21]:
def callfunc(s):
    return  SemiParsedCall(spot, s, rate, volatility, expiry)

targetCall = Constant(19.0, "Target Call", "SEK", 1.0, "SEK", 1.0)

solvedStrike = NewtonRaphson.Solve(targetCall, 119, 1.0e-12, NewtonRaphson.Function(callfunc)).AccumulateDerivatives("SEK", 1.0, constantNumberFormat, constantNumberFormat)

print(solvedStrike.Value)

for derivative in solvedStrike:
    print(derivative)

118.48571148947252
(Constant: 19 (Target Call, SEK, 1, Plain Number Format (1, 2, ., ,, 0), SEK, 1), -1,9629483377678247)
(Constant: 123 (Spot, SEK, 1, Plain Number Format (1, 2, .,  , 0), SEK, 1), 1,2665181293257006)
(Constant: 0,6 (Expiry, Years, 1, Plain Number Format (1, 2, .,  , 0), Day, 0,0027397260273972603), 29,128124834958754)
(Constant: 0,05 (Rate, %, 100, Plain Number Format (1, 2, .,  , 0), bp, 0,01), 71,0914268936835)
(Constant: 0,4 (Volatility, %, 100, Plain Number Format (1, 2, .,  , 0), bp, 0,01), 69,61151778145538)


The result can be formatted:

In [22]:
print("{0:16} : {1:>13}".format("Solved Strike", solvedStrike.FormatValue(True)))

for derivative in solvedStrike:
    print("{0:16} : {1:>13}" .format(derivative.Item1.IdNullable, solvedStrike.FormatDerivative(derivative, True)))

Solved Strike    :    118.49 SEK
Target Call      : -1.96 SEK/SEK
Spot             :  1.27 SEK/SEK
Expiry           :  0.08 SEK/Day
Rate             :   0.71 SEK/bp
Volatility       :   0.70 SEK/bp
